<a href="https://colab.research.google.com/github/Kimtaewookdev/Sec2_project/blob/main/AI_13_%EA%B9%80%ED%83%9C%EC%9A%B1_section2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AI_13 Section2 Project 
# League of Legends 승패 예측

### 1. 데이터 선정 이유 및 문제 정의



In [ ]:
!pip install category_encoders==2.*
!pip install eli5
!pip install --upgrade pandas-profiling
!pip install pdpbox
!pip install shap
!pip install --upgrade xgboost
!pip install --upgrade jinja2

In [179]:
import xgboost
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pandas_profiling import profile_report

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.impute import SimpleImputer
from category_encoders import OrdinalEncoder
from scipy.stats import uniform, randint

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV



warnings.filterwarnings("ignore")
xgboost.set_config(verbosity=0)

### 2. EDA

In [110]:
df = pd.read_csv('/content/drive/MyDrive/sec2 ds/lol_na_s11_highelo.csv')

# 불필요 칼럼 제거

dc1 = df.columns[['veteran' in x for x in df.columns]].tolist()
dc2 = df.columns[['inactive' in x for x in df.columns]].tolist()
dc3 = df.columns[['freshblood' in x for x in df.columns]].tolist()
dc4 = df.columns[['hotstreak' in x for x in df.columns]].tolist()
dc5 = df.columns[['wins' in x for x in df.columns]].tolist()
dc6 = df.columns[['losses' in x for x in df.columns]].tolist()
#dc7 = df.columns[['avg_game_time' in x for x in df.columns]].tolist()

dc = dc1 + dc2 + dc3 + dc4 + dc5 + dc6 #+ dc7

wr = df.copy()

wr.drop(dc, axis = 1, inplace = True)

In [111]:
# 중복 열 확인
wr.duplicated(['gameid']).value_counts()

False    2088
dtype: int64

In [112]:
wr.drop('gameid', axis = 1, inplace = True)

In [113]:
# 티어, 랭크, lp = lp 포인트로 정리
# 결측치는 Simpleimputer mean으로 처리

def trans_rank(wr):
  team = [100, 200]
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for t in team:
    for p in position:
      col = f'{t}_' + p + '_rank'
      wr[col][(wr[col] == 'I')] = -100
      wr[col][(wr[col] == 'II')] = -200
      wr[col][(wr[col] == 'III')] = -300
      wr[col][(wr[col] == 'IV')] = -400
#      wr[col].fillna(0, inplace = True)

def trans_tier(wr):
  team = [100, 200]
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for t in team:
    for p in position:
      col = f'{t}_' + p + '_tier'
      wr[col][(wr[col] == 'CHALLENGER')] = 100
      wr[col][(wr[col] == 'GRANDMASTER')] = 100
      wr[col][(wr[col] == 'MASTER')] = 100
      wr[col][(wr[col] == 'DIAMOND')] = 0
      wr[col][(wr[col] == 'PLATINUM')] = -400
      wr[col][(wr[col] == 'GOLD')] = -800
      wr[col][(wr[col] == 'SILVER')] = -1200
#      wr[col].fillna(0, inplace = True)

def trans_lp(wr):
  team = [100, 200]
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for t in team:
    for p in position:
      tier = f'{t}_' + p + '_tier'
      rank = f'{t}_' + p + '_rank'
      lp = f'{t}_' + p + '_lp'
#      wr[lp].fillna(0, inplace = True)
      wr[lp] = wr[tier] + wr[rank] + wr[lp]
      wr.drop([tier, rank], axis = 1, inplace = True)

trans_rank(wr)
trans_tier(wr)
trans_lp(wr)

wr.shape

(2088, 151)

In [114]:
# 최근 10 게임 포지션 칼럼을 가장 많이 플레이한 포지션 하나로 정리

def mode_is(list):
  return stats.mode(list)[0][0]

def most_position(wr):
  team = [100, 200]
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for t in team:
    for p in position:
      col = f'{t}_' + p + '_positions_played'
      most_col = f'{t}_' + p + '_most_played'
      wr[col] = wr[col].str.split().apply(mode_is)
      wr.rename(columns={col : most_col}, inplace = True)

most_position(wr)

In [117]:
# 데이터를 100(Blue), 200(Red) 팀으로 나누어서 한 팀의 정보만 담은 데이터셋으로 변환

wr_col100 = wr.columns[['100' in x for x in wr.columns]].tolist()
wr_col200 = wr.columns[['200' in x for x in wr.columns]].tolist()

wr_blue = wr[wr_col100].copy()
wr_red = wr[wr_col200].copy()

wt = 'winning_team'

wr_blue[wt] = wr[wt]
wr_red[wt] = wr[wt]

wr_blue['Team'] = 'Blue'
wr_red['Team'] = 'Red'

# 'Result' column은 0 패배 1 승리

wr_blue['Result'] = 0
wr_blue['Result'][wr_blue[wt] == 100] = 1

wr_red['Result'] = 0
wr_red['Result'][wr_red[wt] == 200] = 1

wr_blue.drop(wt, axis = 1, inplace = True)
wr_red.drop(wt, axis = 1, inplace = True)

In [119]:
# colunm 명 변경

def remove_100(x):
  if '100' in x:
    x.remove('100')
  str = '_'.join(x)
  return str

col_united = wr_blue.columns.str.split('_').map(remove_100)

wr_blue.columns = col_united
wr_red.columns = col_united

# 병합

qw = pd.concat([wr_blue, wr_red], ignore_index=True)

In [122]:
# 플레이어가 한 번도 해당 포지션을 해본 적이 없는 경우 게임 수, 승률 0 으로 처리
# 처음 하는 챔피언의 승률 0 으로 처리 (champ_games_played = 0 일때 champ_winrate가 결측치)

#na_qw = qw[qw.columns[['champ' in x for x in qw.columns]]]
#na_col = na_qw.isna().sum(axis = 1)[na_qw.isna().sum(axis = 1) != 0].index
#na_qw.iloc[na_col]

qw_col_role = qw.columns[['role' in x for x in qw.columns]].tolist()
qw_col_champ = qw.columns[['champ' in x for x in qw.columns]].tolist()

for col in qw_col_role + qw_col_champ:
  qw[col].fillna(0, inplace = True)

In [123]:
# 이상치 처리, 평균 게임 시간, 평균 분당 골드획득량에 확실한 이상치 존재
# 게임 시간에는 80분 이상의 데이터, 골드획득량은 기본 획득량이 있음에도 100G 이하의 획득량
# 분당 가한 피해량, 오브젝트 피해량의 경우 지나치게 작은 데이터가 있지만 해당 플레이어들의 lp가 낮아 이상치가 아니라고 판단
# 고의로 트롤링하는 유저의 가능성이 있음.
# 결측치로 처리해서 SimpleImputer로 mean으로 채움.

def outlier_remove(qw):
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for p in position:
    time = p + '_avg_game_time'
    gold = p + '_goldpm'
    qw[time][qw[time] > 80] = np.nan
    qw[gold][qw[gold] < 100] = np.nan

outlier_remove(qw)

qw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4176 entries, 0 to 4175
Data columns (total 77 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   TOP_avg_game_time           4142 non-null   float64
 1   TOP_most_played             4176 non-null   object 
 2   TOP_avg_kda                 4176 non-null   float64
 3   TOP_avg_kp                  4176 non-null   float64
 4   TOP_avg_vision              4176 non-null   float64
 5   TOP_avg_cs                  4176 non-null   float64
 6   TOP_goldpm                  4142 non-null   float64
 7   TOP_dmgpm                   4176 non-null   float64
 8   TOP_dmg_to_obj_pm           4176 non-null   float64
 9   TOP_wr                      4176 non-null   float64
 10  TOP_lp                      3926 non-null   object 
 11  TOP_role_total_played       4176 non-null   float64
 12  TOP_role_wr                 4176 non-null   float64
 13  TOP_champ_games_played      4176 

In [10]:
#pr = qw.profile_report()
#pr.to_file('pr_report.html')

In [124]:
# 훈련 데이터, 테스트 데이터 분리

target = 'Result'

X = qw.drop(target, axis = 1)
y = qw[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

X_train.shape

(3340, 76)

In [12]:
# 시각화

### 3. 모델링 및 모델 해석

In [129]:
# baseline = 0, score = 0.501796

y_train.value_counts(normalize = True)

1    0.501796
0    0.498204
Name: Result, dtype: float64

In [128]:
pipe = make_pipeline(
    OrdinalEncoder()
    , SimpleImputer(strategy='mean')
    , RandomForestClassifier(n_estimators=500, n_jobs=-1)
)

dists = {
    'randomforestclassifier__max_features': uniform(0, 1)
    , 'randomforestclassifier__min_samples_leaf': randint(1,30)
    }

clf = RandomizedSearchCV(
    estimator = pipe
    , param_distributions = dists
    , scoring='accuracy'
    , cv = 4
    , n_iter = 50
    , n_jobs = -1
    )

clf.fit(X_train, y_train)
print("최적 파라미터 :", clf.best_params_)
print("정확도 :", clf.best_score_)

model = clf.best_estimator_

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

최적 파라미터 : {'randomforestclassifier__max_features': 0.14351195259782656, 'randomforestclassifier__min_samples_leaf': 11}
정확도 : 0.7883233532934133
              precision    recall  f1-score   support

           0       0.79      0.83      0.81       424
           1       0.82      0.78      0.80       412

    accuracy                           0.81       836
   macro avg       0.81      0.80      0.80       836
weighted avg       0.81      0.81      0.80       836



In [ ]:
# xgboost

X_xgb, X_val, y_xgb, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=13)

preprocess = make_pipeline(
    OrdinalEncoder()
    , SimpleImputer(strategy='mean')
)

X_xgb_pre = preprocess.fit_transform(X_xgb)
X_val_pre = preprocess.transform(X_val)
X_test_pre = preprocess.transform(X_test)

eval_set = [(X_xgb_pre, y_xgb), (X_val_pre, y_val)]

model_xgb = XGBClassifier(n_estimators = 2000
                          , verbosity = 0
                          , objective = 'binary:logistic'
                          , tree_method='gpu_hist'
                          , gpu_id=0
                          , sub_sample = 0.7
                          , sampling_method = 'gradient_based' 
                          )

param_grid = {'learning_rate' : [0.01, 0.05, 0.075, 0.1, 0.2]
              , 'max_depth' : [4, 5, 6, 7, 8, 9]
              , 'min_child_weight' : [1, 2, 3]
              }

fit_params = {'eval_set' : eval_set
              , 'eval_metric' : 'error'
              , 'early_stopping_rounds' : 100
              , 'verbose' : 0
              }

gcv = GridSearchCV(model_xgb, param_grid = param_grid, cv = 5)

gcv.fit(X_xgb_pre, y_xgb, **fit_params)

# model_xgb.fit(X_xgb_pre, y_xgb, eval_set=eval_set, eval_metric='error', early_stopping_rounds = 200)

In [177]:
model_xgb = gcv.best_estimator_

y_pred_xgb = model_xgb.predict(X_test_pre)

print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.82      0.82      0.82       424
           1       0.81      0.81      0.81       412

    accuracy                           0.81       836
   macro avg       0.81      0.81      0.81       836
weighted avg       0.81      0.81      0.81       836

